# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
file_path = 'data/AviationData.csv'   
#/Users/austinmohn/Documents/Flatiron School/Course 2 Into to Data Science/dsc-course2-m8-lab//data/AviationData.csv'

aviation_df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Check for NaN values
print("Missing Values (NaNs) per column:\n", aviation_df.isna().sum())

# Inspect data types
print("\nData Types:\n", aviation_df.dtypes)

# Summary statistics
print("\nSummary Statistics:\n", aviation_df.describe(include="all"))

/var/folders/6j/5730h4y12159227rqs8wg2r80000gn/T/ipykernel_22636/599939402.py:4: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_df = pd.read_csv(file_path, encoding="ISO-8859-1")


Missing Values (NaNs) per column:
 Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publi

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
# Make a copy of aviation_df to protect the original data
aviation_df_filtered = aviation_df.copy()

# Convert 'Publication.Date' to datetime format
aviation_df_filtered['Event.Date'] = pd.to_datetime(aviation_df_filtered['Event.Date'], errors='coerce')

# Filter out rows where 'Publication.Date' is before 1983
aviation_df_filtered = aviation_df_filtered[aviation_df_filtered['Event.Date'].dt.year >= 1983]

# Filter out amateur built planes
aviation_df_filtered = aviation_df_filtered[aviation_df_filtered['Amateur.Built'] != 'Yes']

# Filter for just planes
aviation_df_filtered['Aircraft.Category'] = aviation_df_filtered.groupby('Model')['Aircraft.Category'].transform(lambda x: x.fillna(x.iloc[0]))

aviation_df_filtered = aviation_df_filtered[aviation_df_filtered['Aircraft.Category'] == 'Airplane']

# Filter irrelevant data
aviation_df_filtered = aviation_df_filtered.drop(columns=['Latitude','Longitude'], axis=1)

# Impute required columns
aviation_df_filtered = aviation_df_filtered.fillna({'Total.Fatal.Injuries': 0, 'Total.Serious.Injuries': 0, 'Total.Minor.Injuries' : 0,	'Total.Uninjured' : 0, 'Aircraft.damage' : 'None'})

# Display the first few rows to verify
aviation_df_filtered.head()



/var/folders/6j/5730h4y12159227rqs8wg2r80000gn/T/ipykernel_22636/153374951.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aviation_df_filtered['Aircraft.Category'] = aviation_df_filtered.groupby('Model')['Aircraft.Category'].transform(lambda x: x.fillna(x.iloc[0]))


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
4149,20001214X42478,Incident,LAX83IA149B,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,Minor,...,Unknown,NaN,0.0,0.0,0.0,588.0,VMC,Standing,Probable Cause,04-12-2014
4150,20001214X42478,Incident,LAX83IA149A,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,Minor,...,NaN,"Singapore Airlines, Ltd.",0.0,0.0,0.0,588.0,VMC,Taxi,Probable Cause,04-12-2014
4171,20001214X42331,Accident,ATL83FA140,1983-03-20,"CROSSVILLE, TN",United States,NaN,NaN,Fatal(1),Destroyed,...,Personal,NaN,1.0,1.0,0.0,0.0,IMC,Cruise,Probable Cause,02-05-2011
4285,20001214X42672,Accident,FTW83LA177,1983-04-02,"MCKINNEY, TX",United States,TX05,AERO COUNTRY,Fatal(1),None,...,Skydiving,NaN,1.0,0.0,0.0,4.0,VMC,Standing,Probable Cause,17-10-2016
4580,20001214X43070,Incident,MIA83AA136,1983-05-05,"MIAMI, FL",United States,MIA,MIAMI INTL,Incident,None,...,Unknown,NaN,0.0,0.0,0.0,172.0,VMC,Descent,Probable Cause,NaN


In [4]:
len(aviation_df_filtered)

21630

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

Let's start with standardizing formatting


In [5]:
# Convert Aircraft.damage to lowercase, stripped
# Standardizing 'Aircraft.damage' column (convert to lowercase and strip spaces)
aviation_df_filtered["Aircraft.damage"] = aviation_df_filtered["Aircraft.damage"].str.lower().str.strip()

# Standardize injury columns to integers
#aviation_df_filtered["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"] = int(aviation_df_filtered["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"])
injury_cols = ["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"]
aviation_df_filtered[injury_cols] = (aviation_df_filtered[injury_cols]).astype(int)


#print confirmation
aviation_df_filtered.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
4149,20001214X42478,Incident,LAX83IA149B,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,minor,...,Unknown,NaN,0,0,0,588,VMC,Standing,Probable Cause,04-12-2014
4150,20001214X42478,Incident,LAX83IA149A,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,minor,...,NaN,"Singapore Airlines, Ltd.",0,0,0,588,VMC,Taxi,Probable Cause,04-12-2014
4171,20001214X42331,Accident,ATL83FA140,1983-03-20,"CROSSVILLE, TN",United States,NaN,NaN,Fatal(1),destroyed,...,Personal,NaN,1,1,0,0,IMC,Cruise,Probable Cause,02-05-2011
4285,20001214X42672,Accident,FTW83LA177,1983-04-02,"MCKINNEY, TX",United States,TX05,AERO COUNTRY,Fatal(1),none,...,Skydiving,NaN,1,0,0,4,VMC,Standing,Probable Cause,17-10-2016
4580,20001214X43070,Incident,MIA83AA136,1983-05-05,"MIAMI, FL",United States,MIA,MIAMI INTL,Incident,none,...,Unknown,NaN,0,0,0,172,VMC,Descent,Probable Cause,NaN


Making the metrics

In [6]:
# Total number of injuries (excluding uninjured passengers)
aviation_df_filtered["Total.Injuries"] = (
    aviation_df_filtered["Total.Fatal.Injuries"] + 
    aviation_df_filtered["Total.Serious.Injuries"] + 
    aviation_df_filtered["Total.Minor.Injuries"]
)

# Estimate total passengers (sum of all reported injury categories and uninjured)
aviation_df_filtered["Estimated.Total.Passengers"] = (
    aviation_df_filtered["Total.Injuries"] + aviation_df_filtered["Total.Uninjured"]
)

# Prevent division by zero
aviation_df_filtered["Estimated.Total.Passengers"] = aviation_df_filtered["Estimated.Total.Passengers"].replace(0, pd.NA)

# Metric: Likelihood of Fatal/Serious Injury
aviation_df_filtered["Serious.Fatal.Injury.Rate"] = (
    (aviation_df_filtered["Total.Fatal.Injuries"] + aviation_df_filtered["Total.Serious.Injuries"]) / 
    aviation_df_filtered["Estimated.Total.Passengers"]
) # There's a lot of 1.0 outputs here. 

# Display the first few rows of the cleaned and enhanced dataset
aviation_df_filtered.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Injuries,Estimated.Total.Passengers,Serious.Fatal.Injury.Rate
4149,20001214X42478,Incident,LAX83IA149B,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,minor,...,0,0,588,VMC,Standing,Probable Cause,04-12-2014,0,588,0.0
4150,20001214X42478,Incident,LAX83IA149A,1983-03-18,"LOS ANGELES, CA",United States,LAX,LOS ANGELES INTL,Incident,minor,...,0,0,588,VMC,Taxi,Probable Cause,04-12-2014,0,588,0.0
4171,20001214X42331,Accident,ATL83FA140,1983-03-20,"CROSSVILLE, TN",United States,NaN,NaN,Fatal(1),destroyed,...,1,0,0,IMC,Cruise,Probable Cause,02-05-2011,2,2,1.0
4285,20001214X42672,Accident,FTW83LA177,1983-04-02,"MCKINNEY, TX",United States,TX05,AERO COUNTRY,Fatal(1),none,...,0,0,4,VMC,Standing,Probable Cause,17-10-2016,1,5,0.2
4580,20001214X43070,Incident,MIA83AA136,1983-05-05,"MIAMI, FL",United States,MIA,MIAMI INTL,Incident,none,...,0,0,172,VMC,Descent,Probable Cause,NaN,0,172,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [7]:
# Column formatting has already been standardized
# Create binary column: 1 if destroyed, 0 otherwise. This will all math functions later
aviation_df_filtered["Aircraft.Destroyed"] = aviation_df_filtered["Aircraft.damage"].apply(lambda x: 1 if x == "destroyed" else 0)

aviation_df_filtered[["Aircraft.damage", "Aircraft.Destroyed"]].head()


,Aircraft.damage,Aircraft.Destroyed
4149,minor,0
4150,minor,0
4171,destroyed,1
4285,none,0
4580,none,0


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

Let's clean up these data entry inconsistencies..

-Convert text to uppercase for consistency.
-Strip extra whitespaces.

Handle Variants & Typos
-Remove inconsistencies (e.g., "BOEING " vs. "Boeing").
-Group similar manufacturer names together if needed.

Filter Out Rare Makes
-Keep only makes with at least 50 occurrences (or adjust as needed).

In [8]:

# Standardize 'Make' column: Uppercase & strip whitespace
aviation_df_filtered["Make"] = aviation_df_filtered["Make"].str.upper().str.strip()

# Standardize 'Make' names (Bombardier vs Bombardier Inc)
def update_make_names(df):
    """
    Function to clean and standardize 'Make' names based on a dictionary of replacements.
    
    Parameters:
        df (DataFrame): The DataFrame containing the 'Make' column.
    
    Returns:
        DataFrame: The DataFrame with updated 'Make' names.
    """
    # Dictionary of replacements
    make_replacements = {
        "AIR TRACTOR INC": "AIR TRACKER",
        "AIRBUS INDUSTRIE": "AIRBUS",
        "AVIAT AIRCRAFT INC": "AVIAT",
        "BOMBARDIER INC": "BOMBARDIER",
        "CIRRUS DESIGN CORP": "CIRRUS",
        "CIRRUS DESIGN CORP.": "CIRRUS",
        "DE HAVILLAND": "DEHAVILLAND",
        "ERCOUPE (ENG & RESEARCH CORP.)": "ERCOUPE",
        "GRUMMAN AMERICAN AVN. CORP.": "GRUMMAN AMERICAN",
        "GRUMMAN ACFT ENG COR-SCHWEIZER" : "GRUMMAN-SCHWEIZER",
        "ROBINSON HELICOPTER": "ROBINSON",
        "ROBINSON HELICOPTER COMPANY": "ROBINSON",
        "ROCKWELL INTERNATIONAL": "ROCKWELL"
    } # Several potentional replacements were left alone due to uncertainty
    #Those include BOEING, GRUMMAN-SCHWEIZER, and AMERICAN
    
    # Apply the replacements
    df["Make"] = df["Make"].replace(make_replacements)
    
    return df

# Apply function to the dataset
aviation_df_filtered = update_make_names(aviation_df_filtered)

# Count occurrences of each Make
make_counts = aviation_df_filtered["Make"].value_counts()

# Filter Makes appearing at least 50 times
valid_makes = make_counts[make_counts >= 50].index
aviation_df_filtered = aviation_df_filtered[aviation_df_filtered["Make"].isin(valid_makes)]

# Display first few rows to verify
aviation_df_filtered["Make"]


4150                         BOEING
4171                          PIPER
4285                    DEHAVILLAND
6336                    DEHAVILLAND
6760                         BOEING
                    ...            
88873                        CIRRUS
88877                        CESSNA
88879                        AIRBUS
88883                   AIR TRACTOR
88886    AMERICAN CHAMPION AIRCRAFT
Name: Make, Length: 18299, dtype: object

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [9]:
# Remove NaNs from 'Model' column
aviation_df_filtered = aviation_df_filtered.dropna(subset=["Model"])

# Inspect the uniqueness of models across different makes
model_counts = aviation_df_filtered.groupby("Model")["Make"].nunique()

# Identify models that appear under multiple makes
non_unique_models = model_counts[model_counts > 1]

# Create a unique identifier if models are not unique per make
if not non_unique_models.empty:
    aviation_df_filtered["Unique_Aircraft_Type"] = aviation_df_filtered["Make"] + " " + aviation_df_filtered["Model"]
else:
    aviation_df_filtered["Unique_Aircraft_Type"] = aviation_df_filtered["Model"]

# Display sample results

aviation_df_filtered[["Make", "Model", "Unique_Aircraft_Type"]].head(10)

,Make,Model,Unique_Aircraft_Type
4150,BOEING,747,BOEING 747
4171,PIPER,PA-28-140,PIPER PA-28-140
4285,DEHAVILLAND,DHC-6,DEHAVILLAND DHC-6
6336,DEHAVILLAND,DHC-6,DEHAVILLAND DHC-6
6760,BOEING,727-200,BOEING 727-200
6806,BEECH,C35,BEECH C35
7084,CESSNA,180K,CESSNA 180K
7708,BEECH,99,BEECH 99
8207,BOEING,747,BOEING 747
8585,PIPER,PA-23-250,PIPER PA-23-250


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

If we're not worried about imputing or dropping NaNs, then I'm going to focus on formatting. The only column that may need to impute NaNs is Number.of.Engines as it's an integer, but we'll see if it's required later in the analysis.

In [10]:
# Standardize 'Engine.Type' column
aviation_df_filtered["Engine.Type"] = aviation_df_filtered["Engine.Type"].str.lower().str.strip()

# Standardize 'Weather.Condition' column (ensure values are either 'vmc' or 'imc')
aviation_df_filtered["Weather.Condition"] = aviation_df_filtered["Weather.Condition"].str.lower().str.strip()

# Convert 'Number.of.Engines' to integer where possible
aviation_df_filtered["Number.of.Engines"] = pd.to_numeric(aviation_df_filtered["Number.of.Engines"], errors="coerce")

# Standardize 'Broad.Phase.of.Flight' column
aviation_df_filtered["Broad.phase.of.flight"] = aviation_df_filtered["Broad.phase.of.flight"].str.lower().str.strip()

# Standardize 'Purpose.of.Flight' column
aviation_df_filtered["Purpose.of.flight"] = aviation_df_filtered["Purpose.of.flight"].str.lower().str.strip()

# Standardize data entry for 'Purpose.of.flight'
def update_purpose_names(df):
    """
    Function to clean and standardize 'Purpose.of.flight' names based on a dictionary of replacements.
    
    Parameters:
        df (DataFrame): The DataFrame containing the 'Purpose.of.flight' column.
    
    Returns:
        DataFrame: The DataFrame with updated 'Purpose.of.flight' names.
    """
    # Dictionary of replacements
    make_replacements = {
        "asho": "air race show",
        "air race/show": "air race show"
    } # Did some research on 'asho' and it means 'air race show'
    
    # Apply the replacements
    df["Purpose.of.flight"] = df["Purpose.of.flight"].replace(make_replacements)
    
    return df

# Apply function to the dataset
aviation_df_filtered = update_purpose_names(aviation_df_filtered)

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [11]:
# Identify existing NaNs
print(aviation_df_filtered.isna().sum())

# Identify total existing rows to compare NaN values to
print(len(aviation_df_filtered))

#Remove rows
aviation_df_filtered = aviation_df_filtered.drop(columns=['Investigation.Type', 
                                                          'Accident.Number',
                                                          'Location',
                                                          'Airport.Code',
                                                          'Airport.Name',
                                                          'Aircraft.Category',
                                                          'Registration.Number',
                                                          'FAR.Description',
                                                          'Schedule',
                                                          'Air.carrier',
                                                          'Broad.phase.of.flight',
                                                          'Report.Status',
                                                          'Publication.Date'], axis=1) 
#I came back to this section after completing Part 2 and removed many additional columns that weren't used for the remainder of the lab.

Event.Id                          0
Investigation.Type                0
Accident.Number                   0
Event.Date                        0
Location                          5
Country                           2
Airport.Code                   6423
Airport.Name                   6320
Injury.Severity                 730
Aircraft.damage                   0
Aircraft.Category                 0
Registration.Number             209
Make                              0
Model                             0
Amateur.Built                    16
Number.of.Engines              2177
Engine.Type                    3308
FAR.Description                 465
Schedule                      16013
Purpose.of.flight              3172
Air.carrier                    9686
Total.Fatal.Injuries              0
Total.Serious.Injuries            0
Total.Minor.Injuries              0
Total.Uninjured                   0
Weather.Condition              2494
Broad.phase.of.flight         15701
Report.Status               

In [12]:
#Confirm rows remaining and NaN count
print(aviation_df_filtered.isna().sum())

Event.Id                         0
Event.Date                       0
Country                          2
Injury.Severity                730
Aircraft.damage                  0
Make                             0
Model                            0
Amateur.Built                   16
Number.of.Engines             2177
Engine.Type                   3308
Purpose.of.flight             3172
Total.Fatal.Injuries             0
Total.Serious.Injuries           0
Total.Minor.Injuries             0
Total.Uninjured                  0
Weather.Condition             2494
Total.Injuries                   0
Estimated.Total.Passengers     810
Serious.Fatal.Injury.Rate      810
Aircraft.Destroyed               0
Unique_Aircraft_Type             0
dtype: int64


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [14]:
aviation_df_filtered.to_csv("cleaned_aviation_data.csv", index=False)